# L5: Prompt Compression


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
#pip install llmlingua

In [3]:
import custom_utils 

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils</code> files:</b> To access <code>requirements.txt</code> for this notebook, 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Data Loading

In [4]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("MongoDB/airbnb_embeddings", streaming=True, split="train")
dataset = dataset.take(100)
# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset)
dataset_df.head(5)

,_id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,...,images,host,address,availability,review_scores,reviews,weekly_price,monthly_price,text_embeddings,image_embeddings
0,10006546,https://www.airbnb.com/rooms/10006546,Ribeira Charming Duplex,Fantastic duplex apartment with three bedrooms...,Privileged views of the Douro River and Ribeir...,Fantastic duplex apartment with three bedrooms...,"In the neighborhood of the river, you can find...",Lose yourself in the narrow streets and stairc...,Transport: • Metro station and S. Bento railwa...,We are always available to help guests. The ho...,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '51399391', 'host_url': 'https://w...","{'street': 'Porto, Porto, Portugal', 'suburb':...","{'availability_30': 28, 'availability_60': 47,...","{'review_scores_accuracy': 9, 'review_scores_c...","[{'_id': '58663741', 'date': 2016-01-03 05:00:...",NaN,NaN,"[0.0123710884, -0.0180913936, -0.016843712, -0...","[-0.1302358955, 0.1534578055, 0.0199299306, -0..."
1,10021707,https://www.airbnb.com/rooms/10021707,Private Room in Bushwick,Here exists a very cozy room for rent in a sha...,,Here exists a very cozy room for rent in a sha...,,,,,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '11275734', 'host_url': 'https://w...","{'street': 'Brooklyn, NY, United States', 'sub...","{'availability_30': 0, 'availability_60': 0, '...","{'review_scores_accuracy': 10, 'review_scores_...","[{'_id': '61050713', 'date': 2016-01-31 05:00:...",NaN,NaN,"[0.0153845912, -0.0348115042, -0.0093448907, 0...","[0.0340401195, 0.1742489338, -0.1572628617, 0...."
2,1001265,https://www.airbnb.com/rooms/1001265,Ocean View Waikiki Marina w/prkg,A short distance from Honolulu's billion dolla...,Great studio located on Ala Moana across the s...,A short distance from Honolulu's billion dolla...,You can breath ocean as well as aloha.,,Honolulu does have a very good air conditioned...,"Pool, hot tub and tennis",...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '5448114', 'host_url': 'https://ww...","{'street': 'Honolulu, HI, United States', 'sub...","{'availability_30': 16, 'availability_60': 46,...","{'review_scores_accuracy': 9, 'review_scores_c...","[{'_id': '4765259', 'date': 2013-05-24 04:00:0...",650.0,2150.0,"[-0.0400562622, -0.0405789167, 0.000644172, 0....","[-0.1640156209, 0.1256971657, 0.6594450474, -0..."
3,10009999,https://www.airbnb.com/rooms/10009999,Horto flat with small garden,One bedroom + sofa-bed in quiet and bucolic ne...,Lovely one bedroom + sofa-bed in the living ro...,One bedroom + sofa-bed in quiet and bucolic ne...,This charming ground floor flat is located in ...,"There´s a table in the living room now, that d...","Easy access to transport (bus, taxi, car) and ...",,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '1282196', 'host_url': 'https://ww...","{'street': 'Rio de Janeiro, Rio de Janeiro, Br...","{'availability_30': 0, 'availability_60': 0, '...","{'review_scores_accuracy': None, 'review_score...",[],1492.0,4849.0,"[-0.063234821, 0.0017937823, -0.0243996996, -0...","[-0.1292964518, 0.037789464, 0.2443587631, 0.0..."
4,10047964,https://www.airbnb.com/rooms/10047964,Charming Flat in Downtown Moda,Fully furnished 3+1 flat decorated with vintag...,The apartment is composed of 1 big bedroom wit...,Fully furnished 3+1 flat decorated with vintag...,With its diversity Moda- Kadikoy is one of the...,,,,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '1241644', 'host_url': 'https://ww...","{'street': 'Kadıköy, İstanbul, Turkey', 'subur...","{'availability_30': 27, 'availability_60': 57,...","{'review_scores_accuracy': 10, 'review_scores_...","[{'_id': '68162172', 'date': 2016-04-02 04:00:...",NaN,NaN,"[0.023723349, 0.0064210771, -0.0339970738, -0....","[-0.1006749049, 0.4022984803, -0.1821258366, 0..."


In [5]:
print("Columns:", dataset_df.columns)

Columns: Index(['_id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'property_type', 'room_type', 'bed_type',
       'minimum_nights', 'maximum_nights', 'cancellation_policy',
       'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review',
       'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'guests_included', 'images', 'host', 'address',
       'availability', 'review_scores', 'reviews', 'weekly_price',
       'monthly_price', 'text_embeddings', 'image_embeddings'],
      dtype='object')


## Document Modelling

In [6]:
listings = custom_utils.process_records(dataset_df)

## Database Creation and Connection

In [7]:
db, collection = custom_utils.connect_to_database()

Connection to MongoDB successful


In [8]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000002'), 'opTime': {'ts': Timestamp(1728311142, 1), 't': 2}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728311142, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1728311142, 1)}, acknowledged=True)

## Data Ingestion

In [9]:
collection.insert_many(listings)
print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


## Vector Search Index defintion

In [10]:
# Create vector search index
custom_utils.setup_vector_search_index_with_filter(collection=collection)

Creating index...
New index created successfully: vector_index_with_filter


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note:</b> If the output of the previous cell is <code>Error creating vector search index: Duplicate Index</code> you may proceed to the next cell if you intend to still use a previously created index.</p>

## Handling User Query

In [11]:
from pydantic import BaseModel
from typing import Optional

class SearchResultItem(BaseModel):
    name: str
    accommodates: Optional[int] = None
    address: custom_utils.Address
    neighborhood_overview: Optional[str] = None
    notes: Optional[str] = None
    averageReviewScore: Optional[float] = None
    number_of_reviews: Optional[float] = None
    combinedScore: Optional[float] = None

## Boosting Search Results After Vector Search

In [12]:
review_average_stage = {
    "$addFields": {
        "averageReviewScore": {
            "$divide": [
                {
                    "$add": [
                        "$review_scores.review_scores_accuracy",
                        "$review_scores.review_scores_cleanliness",
                        "$review_scores.review_scores_checkin",
                        "$review_scores.review_scores_communication",
                        "$review_scores.review_scores_location",
                        "$review_scores.review_scores_value",
                    ]
                },
                6  # Divide by the number of review score types to get the average
            ]
        },
        # Calculate a score boost factor based on the number of reviews
        "reviewCountBoost": "$number_of_reviews"
    }
}

In [13]:
weighting_stage = {
    "$addFields": {
        "combinedScore": {
            # Example formula that combines average review score and review count boost
            "$add": [
                {"$multiply": ["$averageReviewScore", 0.3]},  # Weighted average review score
                {"$multiply": ["$reviewCountBoost", 0.7]}   # Weighted review count boost
            ]
        }
    }
}

In [14]:
# Apply the combinedScore for sorting
sorting_stage_sort = {
    "$sort": {"combinedScore": -1}  # Descending order to boost higher combined scores
}

In [15]:
additional_stages = [review_average_stage, weighting_stage, sorting_stage_sort]

## Modified Handling User Query

In [16]:
from IPython.display import display, HTML
import pprint

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    get_knowledge = custom_utils.vector_search_with_filter(query, db, collection, stages, vector_index)

    if not get_knowledge:
        return "No results found.", "No source information available."
    
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    print("Uncompressed Prompt (Query Info):\n")
    print(search_results_df)

    completion = custom_utils.openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "You are a airbnb listing recommendation system."},
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )
    system_response = completion.choices[0].message.content
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")
    display(HTML(search_results_df.to_html()))
    return system_response

In [34]:
# FIX #19

import json
from bson import json_util
from custom_utils import get_embedding

# def debug_print(label, obj):
#     print(f"\n--- Debug: {label} ---")
#     if isinstance(obj, dict):
#         for key, value in obj.items():
#             print(f"{key}: {type(value)}")
#     else:
#         print(f"Type: {type(obj)}")
#         print(f"Value: {obj}")

def vector_search_with_filter(user_query, db, collection, additional_stages=[], vector_index="vector_index_text"):
    query_embedding = get_embedding(user_query)
    # debug_print("Query Embedding", query_embedding)
    if query_embedding is None:
        return "Error generating embedding for the query."

    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index,
            "queryVector": query_embedding,
            "path": "text_embeddings",
            "numCandidates": 150,
            "limit": 20,
            "filter": {
                "$and": [
                    {"accommodates": {"$gte": 2}}, 
                    {"bedrooms": {"$lte": 7}}
                ]
            },
        }
    }
    # debug_print("Vector Search Stage", vector_search_stage)
    
    pipeline = [vector_search_stage] + additional_stages
    # debug_print("Full Pipeline", pipeline)
    
    try:
        results = list(collection.aggregate(pipeline))
        # debug_print("Search Results", results)

        explain_query_execution = db.command( 
            'explain', 
            {
                'aggregate': collection.name,
                'pipeline': pipeline,
                'cursor': {}
            },
            verbosity='executionStats'
        )
        
        print("\nFull Explain Output:")
        # print(json.dumps(explain_query_execution, indent=2, default=json_util.default))
        
        stages = explain_query_execution.get('stages', [])
        print(f"\nStages found in explain output: {len(stages)}")
        for i, stage in enumerate(stages):
            print(f"\n--- Stage {i} ---")
            # print(json.dumps(stage, indent=2, default=json_util.default))
        
        vector_search_explain = next((stage for stage in stages if '$vectorSearch' in stage), {}).get('$vectorSearch', {})
        print("\nVector Search Explain Section:")
        # print(json.dumps(vector_search_explain, indent=2, default=json_util.default))
        
        collect_stats = vector_search_explain.get('explain', {}).get('collectStats', {})
        print("\nCollect Stats Section:")
        # print(json.dumps(collect_stats, indent=2, default=json_util.default))
        
        millis_elapsed = collect_stats.get('millisElapsed', 'N/A')
        print(f"\nTotal time for the execution to complete on the database server: {millis_elapsed} milliseconds")

    except Exception as e:
        print(f"Error during vector search: {str(e)}")
        return f"Error during vector search: {str(e)}"
    
    return results

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    try:
        # debug_print("Input Query", query)
        # debug_print("Vector Index", vector_index)
        
        get_knowledge = vector_search_with_filter(query, db, collection, stages, vector_index)
        # debug_print("Get Knowledge Result", get_knowledge)
        
        if isinstance(get_knowledge, str):  # Error message
            return get_knowledge, "No source information available."
        
        if not get_knowledge:
            return "No results found.", "No source information available."
        
        first_result = get_knowledge[0] if get_knowledge else None
        return str(first_result), "Source: Vector search results"

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return "An error occurred while processing your query.", "No source information available."

# Usage
query = """
I want to stay in a place that's warm and friendly, 
and not too far from restaurants, can you recommend a place? 
Include a reason as to why you've chosen your selection"
"""

result, source = handle_user_query(
    query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)

# Convert JSON string to a dictionary
#json_dict = json.loads(json_string)

# Print the dictionary as JSON with indentation
# print(json.dumps(json_dict, indent=2))


# result_dt = json.loads(result)
# print(type(result))
print("Result:", result)
# print(json.dumps(result_dt, indent=2))
print("Source:", source)


Full Explain Output:

Stages found in explain output: 5

--- Stage 0 ---

--- Stage 1 ---

--- Stage 2 ---

--- Stage 3 ---

--- Stage 4 ---

Vector Search Explain Section:

Collect Stats Section:

Total time for the execution to complete on the database server: N/A milliseconds
Result: {'_id': ObjectId('6703ef6af0a7283252f29796'), 'listing_url': 'https://www.airbnb.com/rooms/10140368', 'name': 'A bedroom far away from home', 'summary': 'NOTES: BEFORE BOOKING, PLEASE KNOW THAT WE ARE LOCATED ON THE 3RD FLOOR OF A WALK UP BUILDING (3 SHORT FLIGHT OF (Website hidden by Airbnb) IF YOU HAVE TROUBLE CLIMBING STAIRS, BE ADVISED, THANKS! THE CHECK-IN TIME IS STRICTLY BETWEEN 1PM-10PM.. AND CHECK OUT IS 11AM THIS LISTING IS ONLY FOR A BEDROOM AND NOT THE ENTIRE APARTMENT.', 'space': 'our place is a good sized apartment in a very quiet neighborhood. the bedroom is clean and cozy with a queen sized bed, so you can get all the rest you need.', 'description': 'NOTES: BEFORE BOOKING, PLEASE KNOW T

In [35]:
query = """
I want to stay in a place that's warm and friendly, 
and not too far from resturants, can you recommend a place? 
Include a reason as to why you've chosen your selection"
"""
handle_user_query(
    query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)


Full Explain Output:

Stages found in explain output: 5

--- Stage 0 ---

--- Stage 1 ---

--- Stage 2 ---

--- Stage 3 ---

--- Stage 4 ---

Vector Search Explain Section:

Collect Stats Section:

Total time for the execution to complete on the database server: N/A milliseconds


('{\'_id\': ObjectId(\'6703ef6af0a7283252f29796\'), \'listing_url\': \'https://www.airbnb.com/rooms/10140368\', \'name\': \'A bedroom far away from home\', \'summary\': \'NOTES: BEFORE BOOKING, PLEASE KNOW THAT WE ARE LOCATED ON THE 3RD FLOOR OF A WALK UP BUILDING (3 SHORT FLIGHT OF (Website hidden by Airbnb) IF YOU HAVE TROUBLE CLIMBING STAIRS, BE ADVISED, THANKS! THE CHECK-IN TIME IS STRICTLY BETWEEN 1PM-10PM.. AND CHECK OUT IS 11AM THIS LISTING IS ONLY FOR A BEDROOM AND NOT THE ENTIRE APARTMENT.\', \'space\': \'our place is a good sized apartment in a very quiet neighborhood. the bedroom is clean and cozy with a queen sized bed, so you can get all the rest you need.\', \'description\': \'NOTES: BEFORE BOOKING, PLEASE KNOW THAT WE ARE LOCATED ON THE 3RD FLOOR OF A WALK UP BUILDING (3 SHORT FLIGHT OF (Website hidden by Airbnb) IF YOU HAVE TROUBLE CLIMBING STAIRS, BE ADVISED, THANKS! THE CHECK-IN TIME IS STRICTLY BETWEEN 1PM-10PM.. AND CHECK OUT IS 11AM THIS LISTING IS ONLY FOR A BEDRO

## Prompt Compression

In [36]:
import json
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    model_config={"revision": "main"},
    use_llmlingua2=True,
    device_map="cpu",
)

# Function definition
def compress_query_prompt(query):

    demonstration_str = query['demonstration_str']
    instruction = query['instruction']
    question = query['question']

    # 6x Compression
    compressed_prompt = llm_lingua.compress_prompt(
        demonstration_str.split("\n"), 
        instruction=instruction,
        question=question,
        target_token=500,
        rank_method="longllmlingua", 
        context_budget="+100",
        dynamic_context_compression_ratio=0.4,
        reorder_context="sort",
    )

    return json.dumps(compressed_prompt, indent=4)


config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [37]:
def handle_user_query_with_compression(query, db, collection, stages=[], vector_index="vector_index_text"):
    # Perform vector search to get knowledge from the database
    get_knowledge = custom_utils.vector_search_with_filter(query, db, collection, stages, vector_index)

    # Check if there are any results
    if not get_knowledge:
        return None, "No results found."

    # Convert search results into a list of SearchResultItem models
    search_results_models = [SearchResultItem(**result) for result in get_knowledge]

    # Convert search results into a DataFrame for better rendering
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # Prepare information for compression
    query_info = {
        'demonstration_str': search_results_df.to_string(),  # Results from information retrieval process
        'instruction': "Write a high-quality answer for the given question using only the provided search results.",
        'question': query  # User query
    }

    # Compress the query prompt using predefined function
    compressed_prompt = compress_query_prompt(query_info)

    # Optional: Print compressed prompts for debugging
    print("Compressed Prompt:\n")
    pprint.pprint(compressed_prompt)
    print("\n" + "=" * 80 + "\n")

    return search_results_df, compressed_prompt



In [38]:
def handle_system_response(query, compressed_prompt):
    # Generate system response using OpenAI's completion
    completion = custom_utils.openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an Airbnb listing recommendation system."
            },
            {
                "role": "user",
                "content": f"Answer this user query: {query} with the following context:\n{compressed_prompt}"
            }
        ]
    )

    system_response = completion.choices[0].message.content

    # Print User Question, System Response
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # Return the system response
    return system_response


In [39]:
# Compress the query and get search results
results, compressed_prompt = handle_user_query_with_compression(query, 
                                                                  db, 
                                                                  collection, 
                                                                  additional_stages, 
                                                                  vector_index="vector_index_with_filter"
                                                                 )

KeyError: 'millisElapsed'

In [40]:
# FIX #1

import json
from bson import json_util
from custom_utils import get_embedding

def vector_search_with_filter(user_query, db, collection, additional_stages=[], vector_index="vector_index_text"):
    query_embedding = get_embedding(user_query)
    if query_embedding is None:
        return "Error generating embedding for the query."

    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index,
            "queryVector": query_embedding,
            "path": "text_embeddings",
            "numCandidates": 150,
            "limit": 20,
            "filter": {
                "$and": [
                    {"accommodates": {"$gte": 2}}, 
                    {"bedrooms": {"$lte": 7}}
                ]
            },
        }
    }
    
    pipeline = [vector_search_stage] + additional_stages
    
    try:
        results = list(collection.aggregate(pipeline))

        explain_query_execution = db.command( 
            'explain', 
            {
                'aggregate': collection.name,
                'pipeline': pipeline,
                'cursor': {}
            },
            verbosity='executionStats'
        )
        
        print("\nFull Explain Output:")
        print(json.dumps(explain_query_execution, indent=2, default=json_util.default))
        
        stages = explain_query_execution.get('stages', [])
        print(f"\nStages found in explain output: {len(stages)}")
        for i, stage in enumerate(stages):
            print(f"\n--- Stage {i} ---")
            print(json.dumps(stage, indent=2, default=json_util.default))
        
        vector_search_explain = next((stage for stage in stages if '$vectorSearch' in stage), {}).get('$vectorSearch', {})
        print("\nVector Search Explain Section:")
        print(json.dumps(vector_search_explain, indent=2, default=json_util.default))
        
        collect_stats = vector_search_explain.get('explain', {}).get('collectStats', {})
        print("\nCollect Stats Section:")
        print(json.dumps(collect_stats, indent=2, default=json_util.default))
        
        # Instead of directly accessing 'millisElapsed', we'll look for timing information more broadly
        timing_info = collect_stats.get('millisElapsed', 'N/A')
        if timing_info == 'N/A':
            # If 'millisElapsed' is not available, let's check if there's any other timing information
            timing_info = json.dumps({k: v for k, v in collect_stats.items() if 'time' in k.lower() or 'duration' in k.lower()}, indent=2, default=json_util.default)
        
        print(f"\nTiming information for the query execution: {timing_info}")

    except Exception as e:
        print(f"Error during vector search: {str(e)}")
        return f"Error during vector search: {str(e)}"
    
    return results

def handle_user_query_with_compression(query, db, collection, stages=[], vector_index="vector_index_text"):
    # Perform vector search to get knowledge from the database
    get_knowledge = vector_search_with_filter(query, db, collection, stages, vector_index)
    
    # Check if there are any results or if an error occurred
    if isinstance(get_knowledge, str):  # Error occurred
        return None, get_knowledge
    if not get_knowledge:
        return None, "No results found."
    
    # Convert search results into a list of SearchResultItem models
    search_results_models = [SearchResultItem(**result) for result in get_knowledge]
    # Convert search results into a DataFrame for better rendering
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])
    # Prepare information for compression
    query_info = {
        'demonstration_str': search_results_df.to_string(),  # Results from information retrieval process
        'instruction': "Write a high-quality answer for the given question using only the provided search results.",
        'question': query  # User query
    }
    # Compress the query prompt using predefined function
    compressed_prompt = compress_query_prompt(query_info)
    # Optional: Print compressed prompts for debugging
    print("Compressed Prompt:\n")
    pprint.pprint(compressed_prompt)
    print("\n" + "=" * 80 + "\n")
    return search_results_df, compressed_prompt

# Usage
query = """
I want to stay in a place that's warm and friendly, 
and not too far from restaurants, can you recommend a place? 
Include a reason as to why you've chosen your selection"
"""

results, compressed_prompt = handle_user_query_with_compression(
    query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)

if results is not None:
    print("Results:")
    print(results)
    print("\nCompressed Prompt:")
    print(compressed_prompt)
else:
    print("Error or no results:", compressed_prompt)

Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors



Full Explain Output:
{
  "explainVersion": "1",
  "stages": [
    {
      "$vectorSearch": {
        "queryVector": [
          -0.014628283679485321,
          -0.055600717663764954,
          -0.01916900835931301,
          -0.0025450566317886114,
          -0.014972479082643986,
          -0.006963327992707491,
          -0.0449041984975338,
          -0.002255469560623169,
          -0.006460273638367653,
          0.005811598617583513,
          -0.05703045055270195,
          0.020969413220882416,
          -0.04334208369255066,
          -0.007585526444017887,
          -0.008558538742363453,
          0.013582460582256317,
          0.011345193721354008,
          0.04929930344223976,
          0.03531969338655472,
          0.048875678330659866,
          0.01288745179772377,
          0.02655596099793911,
          -0.11120143532752991,
          -0.025668997317552567,
          0.023100774735212326,
          0.02057226561009884,
          -0.027535593137145042,
          0

Compressed Prompt:

('{\n'
 '    "compressed_prompt": "name accommodates neighborhood overview notes '
 'averageReviewScore reviews\\n\\nCozy double bed room \\u6771 \\u6d8c \\u9109 '
 "\\u6751 \\u96c5 \\u7dfb \\u96d9 \\u4eba \\u623f street ' Hong Kong New "
 "Territories Hong Kong government _ area Islands market ' Hong Kong country ' "
 "Hong Kong country _ code HK location ' type Point coordinates 113. 92823, "
 '22. 27671 ] is location _ exact 10 - minute walk bus stop Tung Chung Road '
 'Indian Thai Food restaurants 15 - minute walk Yat Tung Estate shopping '
 'centre restaurants bus terminus 7 - minute Chung Mun Road Mun Tung Estate '
 'Car Park Entrance 10 - minute Airport / Asia Expo taxi car 4 - minute Tung '
 'Chung MTR station taxi car toothbrush 7 - walk village house main road room '
 'G / F No lift building \\u5099 \\u7259 \\u818f \\u7259 \\u5237 \\u62d6 '
 '\\u978b \\u5c4b \\u5b50 \\u9060 \\u96e2 \\u99ac \\u8def \\u5ba2 \\u6b65 '
 '\\u884c \\u4e03 \\u5206 \\u9418 \\u5230

In [ ]:
if compressed_prompt:
    # Handle the system response with the compressed prompt
    system_response = handle_system_response(query, compressed_prompt)
else:
    print("No valid results to display.")

## Additional Resouces

- [The MongoDB Developer Center for tutorials, articles and videos](https://mdb.link/developer_center)

- [The GenAI Showcase Repo for code showcasing building AI applications and demo](https://github.com/mongodb-developer/GenAI-Showcase)

- [DeepLearning AI forum for question in regards to this course](https://community.deeplearning.ai/)